This script contains the code for MICOM for all 3 diseases- Inflammatory Bowel Disease (IBD), Gastric Cancer (GC) and Colorectal Cancer (CC). 

The code is repeated and in seperate cells for each disease due to the long time and processing power it takes to run each model.

### Install all required packages:

In [ ]:
! pip install micom 
! pip install numpy Cython
! pip install biom-format
! pip install wget
! pip install pickleshare

### Import all required libraries and functions:

In [ ]:
#for data manipulation:
import pandas as pd
import numpy as np
import pickle
#for building community:
from micom.workflows import build
from micom import Community
from micom.qiime_formats import load_qiime_medium
from micom.workflows import grow
from micom.workflows import tradeoff
import wget
#for visualizing results:
from micom.viz import plot_growth
from micom.viz import plot_exchanges_per_sample
from micom.viz import plot_exchanges_per_taxon
from micom.viz import plot_association
from micom.measures import production_rates
from micom.viz import plot_mes
from micom.viz import plot_tradeoff

### Import required data:

#### IBD Dataset:

In [ ]:
#create data frame of the genus data:
IBD_genus = 'genera_IBD.csv'
IBD_genus_df = pd.read_csv(format(IBD_genus))
#Extract the disease column:
disease_ibd = list(IBD_genus_df['Group'])
#Remove this from the data frame:
IBD_genus_df = IBD_genus_df.drop('Group', axis = 1)
#Set sample_id the index:
IBD_genus_df.set_index('Sample_ID', inplace=True)

#### Gastric Cancer Dataset:

In [ ]:
#create data frame of the genus data:
GC_genus = 'genera_GC.csv'
GC_genus_df = pd.read_csv(format(GC_genus))
#Extract the disease column:
disease_gc = list(GC_genus_df['Group'])
#Remove this from the data frame:
GC_genus_df = GC_genus_df.drop('Group', axis = 1)
#Set sample_id the index:
GC_genus_df.set_index('Sample_ID', inplace=True)

#### Colorectal Cancer Dataset:

In [ ]:
#create data frame of the genus data:
CC_genus = 'genera_CC.csv'
CC_genus_df = pd.read_csv(format(CC_genus))
#Extract the disease column:
disease_cc = list(CC_genus_df['Group'])
#Remove this from the data frame:
CC_genus_df = CC_genus_df.drop('Group', axis = 1)
#Set sample_id the index:
CC_genus_df.set_index('Sample_ID', inplace=True)

### Make required long form data:

#### IBD Dataset:

In [ ]:
#melt:
dfm_ibd = IBD_genus_df.melt(ignore_index=False).reset_index()
dfm_ibd.rename(columns={'Sample_ID': 'sample_id', 'value': 'relative', 'variable': 'genus'}, inplace=True)
#create required id column for manifest function:
dfm_ibd['id'] = dfm_ibd['genus']
#add disease group column onto the data dataframe:
dfm_ibd['disease_stat'] = disease_ibd*83
#save as new file:
dfm_ibd.to_csv('data_long_form_IBD.csv')

#### Gastric Cancer Dataset:

In [ ]:
#melt:
dfm_gc = GC_genus_df.melt(ignore_index=False).reset_index()
dfm_gc.rename(columns={'Sample_ID': 'sample_id', 'value': 'relative', 'variable': 'genus'}, inplace=True)
#create required id column for manifest function:
dfm_gc['id'] = dfm_gc['genus']
#add disease group column onto the data dataframe:
dfm_gc['disease_stat'] = disease_gc*59
#save as new file:
dfm_gc.to_csv('data_long_form_GC.csv')

#### Colorectal Cancer Dataset:

In [ ]:
#melt:
dfm_cc = CC_genus_df.melt(ignore_index=False).reset_index()
dfm_cc.rename(columns={'Sample_ID': 'sample_id', 'value': 'relative', 'variable': 'genus'}, inplace=True)
#create required id column for manifest column:
dfm_cc['id'] = dfm_cc['genus']
#add disease group column onto the data dataframe:
dfm_cc['disease_stat'] = disease_cc*82
#save as a new file:
dfm_cc.to_csv('data_long_form_CC.csv')

#### The resulting dataframes should have:

Column 1) sample_id - the sample each measure corresponds to

Column 2) genus - the genus being measured

Column 3) relative - the relative abunance of the genus for that sample

Column 4) id- same as genus essentially but required by the model

Column 5) disease_status - if the sample the measure comes from has IBD or not. 

Optional: Column 6) The family corresponding to each genus.

### Obtain required AGORA database and diet data:

In [ ]:
!python -m wget -o agora103_genus.qza https://zenodo.org/record/3755182/files/agora103_genus.qza?download=1
!python -m wget -o western_diet_gut.qza https://zenodo.org/record/3755182/files/western_diet_gut.qza?download=1

## Builiding the manifest:

In [ ]:
manifest_ibd = build(dfm_ibd, "agora103_genus.qza", "gen_mod_ibd", solver="osqp",
                 cutoff=0, threads=2) #gen_mod_ibd is the name of a folder to store the results to 

In [ ]:
manifest_gc = build(dfm_gc, "agora103_genus.qza", "gen_mod_gc", solver="osqp",
                 cutoff=0, threads=2) #gen_mod_gc is the name of a folder to store the results to 

In [ ]:
manifest_cc = build(dfm_cc, "agora103_genus.qza", "gen_mod_cc", solver="osqp",
                 cutoff=0, threads=2) #gen_mod_ibd is the name of a folder to store the results to 

## Finding Optimal Tradeoff:

In [ ]:
#load the diet data:
medium = load_qiime_medium("western_diet_gut.qza")

In [ ]:
#Testing different trade-off values:
#IBD:
tradeoff_results_ibd = tradeoff(manifest_ibd, "gen_mod_ibd", medium, threads=2)
tradeoff_results_ibd.to_csv("tradeoff_ibd.csv", index=False)
#GC:
tradeoff_results_gc = tradeoff(manifest_gc, "gen_mod_gc", medium, threads=2)
tradeoff_results_gc.to_csv("tradeoff_gc.csv", index=False)
#CC:
tradeoff_results_cc = tradeoff(manifest_cc, "gen_mod_cc", medium, threads=2)
tradeoff_results_cc.to_csv("tradeoff_cc.csv", index=False)

## Obtaining the growth model:

Using the optimal tradeoff value as ascertained from above:

In [ ]:
#IBD:
growth_results_ibd = grow(manifest_ibd, "gen_mod_ibd", medium, tradeoff=0.9, threads=2) #store to the same folder
# Save the individual results to a file:
pickle.dump(growth_results_ibd, open("growth.pickle_ibd", "wb"))
#GC:
growth_results_gc = grow(manifest_gc, "gen_mod_gc", medium, tradeoff=0.8, threads=2) #store to the same folder
# Save the individual results to a file:
pickle.dump(growth_results_gc, open("growth.pickle_gc", "wb"))
#CC:
growth_results_cc = grow(manifest_cc, "gen_mod_cc", medium, tradeoff=0.7, threads=2) #store to the same folder
# Save the individual results to a file:
pickle.dump(growth_results_ibd, open("growth.pickle_cc", "wb"))

## Visualising results:

In [ ]:
#Visualize the growth rates:
pl = plot_growth(growth_results_ibd, filename="growth_rates_IBD.html")
pl = plot_growth(growth_results_gc, filename="growth_rates_GC.html")
pl = plot_growth(growth_results_cc, filename="growth_rates_CC.html")

#Visulaize how the metabolites are exchanged between samples:
pl = plot_exchanges_per_sample(growth_results_ibd, filename="consumption_IBD.html")
pl = plot_exchanges_per_sample(growth_results_gc, filename="consumption_GC.html")
pl = plot_exchanges_per_sample(growth_results_cc, filename="consumption_CC.html")

#Visualize how the metabolites are exchaned between the genus:
pl = plot_exchanges_per_taxon(growth_results_ibd, filename="niche_IBD.html")
pl = plot_exchanges_per_taxon(growth_results_gc, filename="niche_GC.html")
pl = plot_exchanges_per_taxon(growth_results_cc, filename="niche_CC.html")

## Testing for significant metabolites:

In [ ]:
#reset the index of the manifest to the sample id:
manifest_ibd.index = manifest_ibd.sample_id
manifest_gc.index = manifest_gc.sample_id
manifest_cc.index = manifest_cc.sample_id

#extract the disease status to use for the phenotype:
pheno_ibd = manifest_ibd.disease_stat
pheno_gc = manifest_gc.disease_stat
pheno_cc = manifest_cc.disease_stat

#find associations between metabolites and disease:
pl = plot_association(growth_results_ibd,phenotype= pheno_ibd,variable_type="binary",filename="association_IBD.html",fdr_threshold=0.5)
pl = plot_association(growth_results_gc,phenotype= pheno_gc,variable_type="binary",filename="association_GC.html",fdr_threshold=0.5)
pl = plot_association(growth_results_cc,phenotype= pheno_cc,variable_type="binary",filename="association_CC.html",fdr_threshold=0.5)